1.ipynb：一份PDF文档，有三个（问题，答案）对 - 简单RAG - Ragas评估

# 加载环境变量

In [2]:
import os
from dotenv import find_dotenv, load_dotenv

# 加载环境变量
load_dotenv(find_dotenv())

True

# 源数据

[2023全球智能汽车AI挑战赛——赛道一：AI大模型检索问答](https://tianchi.aliyun.com/competition/entrance/532154/forum)

# 构建RAG

In [3]:
def pretty_print_docs(docs):
  print(
      f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n + {d.page_content}\n{d.metadata}" for i,d in enumerate(docs)])
  )

In [4]:
from langchain_community.document_loaders import PyPDFLoader

# 加载文档
file_path = "data/初赛训练数据集.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
print(len(docs))

354


In [5]:
pretty_print_docs(docs[:3])

Document 1:

 + 欢迎
感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。
首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。
本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际
接收的车辆为准。
如您有任何问题，或需要预约服务，请拨打电话4006-010101 联系我们。您也可以开车前往Lynk &Co领克中心。
在抵达之前，请您注意驾车安全。©领克汽车销售有限公司
{'source': 'data/初赛训练数据集.pdf', 'page': 0}
----------------------------------------------------------------------------------------------------
Document 2:

 + 
{'source': 'data/初赛训练数据集.pdf', 'page': 1}
----------------------------------------------------------------------------------------------------
Document 3:

 + 3目录
前言
本手册相关的重要信息 .................................................11
敬告用户.................................................................11
联系Lynk&Co领克 .....................................................12
事件数据记录系统 ......................................................12
远程监控系统............................................................12
原厂精装附件、选装装备和改装 ......................................13
无线电设备 ..................

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 分割文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=50,
)
split_docs = text_splitter.split_documents(docs)
print(len(split_docs))

807


In [7]:
pretty_print_docs(split_docs[:3])

Document 1:

 + 欢迎
感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。
首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。
本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际
接收的车辆为准。
如您有任何问题，或需要预约服务，请拨打电话4006-010101 联系我们。您也可以开车前往Lynk &Co领克中心。
在抵达之前，请您注意驾车安全。©领克汽车销售有限公司
{'source': 'data/初赛训练数据集.pdf', 'page': 0}
----------------------------------------------------------------------------------------------------
Document 2:

 + 3目录
前言
本手册相关的重要信息 .................................................11
敬告用户.................................................................11
联系Lynk&Co领克 .....................................................12
{'source': 'data/初赛训练数据集.pdf', 'page': 2}
----------------------------------------------------------------------------------------------------
Document 3:

 + 事件数据记录系统 ......................................................12
远程监控系统............................................................12
原厂精装附件、选装装备和改装 ......................................13
{'source': 'data/初赛训练数据集.

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

# 创建嵌入模型
model_name = 'BAAI/bge-large-zh-v1.5'
model_kwargs = {'device': 'cuda'}  # 需要安装GPU版本的torch 
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from langchain_community.vectorstores import FAISS

# 创建向量存储
vectordb = FAISS.from_documents(split_docs, embeddings)

In [10]:
index_folder_path = "data/faiss_index"
index_name = "1"

In [11]:
# 保存索引
vectordb.save_local(index_folder_path, index_name)

In [12]:
# 加载索引
vectordb = FAISS.load_local(index_folder_path, embeddings, index_name)

In [13]:
# 创建检索器
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [14]:
from langchain_openai import ChatOpenAI

# 创建模型
llm = ChatOpenAI(temperature=0)
print(llm.model_name)

gpt-3.5-turbo


In [15]:
from langchain.chains import RetrievalQA

# 创建链
chain = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

In [16]:
# 运行链 - 测试
question = "怎么打开危险警告灯？"
response = chain(question)
print(response)

{'query': '怎么打开危险警告灯？', 'result': '要打开危险警告灯，您需要按下车辆上的危险警告灯按键。当车辆遇到交通事故或其他紧急情况时，按下该按键即可启用危险警告灯。值得注意的是，在发生碰撞情况下，危险警告灯也会自动点亮。', 'source_documents': [Document(page_content='使用危险警告灯 ........................................................ 89', metadata={'source': 'data/初赛训练数据集.pdf', 'page': 3}), Document(page_content='指示灯闪烁。打开危险警告灯开关时，两侧转\n向指示灯均闪烁。\n近光灯指示灯：打开近光灯时，该指示灯点亮。\n远光灯指示灯：打开远光灯时，该指示灯点亮。\n智能远近光控制开启指示灯：打开智能远近光控制时，该\n指示灯点亮。\n后雾灯指示灯：打开后雾灯时，该指示灯点亮。\n位置灯指示灯：打开位置灯时，该指示灯点亮。\n泊车紧急制动指示灯：泊车紧急制动激活时，该指示灯点\n亮。', metadata={'source': 'data/初赛训练数据集.pdf', 'page': 73}), Document(page_content='灯点亮。\n陡坡缓降系统故障警告灯：陡坡缓降系统出现故障时，该\n警告灯点亮。\n安全气囊故障警告灯：安全气囊系统或预紧器系统存在故\n障时，该警告灯点亮。', metadata={'source': 'data/初赛训练数据集.pdf', 'page': 72}), Document(page_content='仪表和灯光\n89\n01点击开启/关闭门控灯。\n使用超车灯\n超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一\n次，以提醒前方车辆。\n使用危险警告灯\n危险警告灯按键\n车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危\n险警告灯。\n说明！\n□发生碰撞情况下，危险警告灯也将自动点亮。', metadata={'source': 'data/初赛训练数据集.pdf', 'page': 88}), Document(page_content='灯常亮。盲点监测系统标定未完成时，该警

In [17]:
response['query']

'怎么打开危险警告灯？'

In [18]:
response['result']

'要打开危险警告灯，您需要按下车辆上的危险警告灯按键。当车辆遇到交通事故或其他紧急情况时，按下该按键即可启用危险警告灯。值得注意的是，在发生碰撞情况下，危险警告灯也会自动点亮。'

In [19]:
pretty_print_docs(response['source_documents'])

Document 1:

 + 使用危险警告灯 ........................................................ 89
{'source': 'data/初赛训练数据集.pdf', 'page': 3}
----------------------------------------------------------------------------------------------------
Document 2:

 + 指示灯闪烁。打开危险警告灯开关时，两侧转
向指示灯均闪烁。
近光灯指示灯：打开近光灯时，该指示灯点亮。
远光灯指示灯：打开远光灯时，该指示灯点亮。
智能远近光控制开启指示灯：打开智能远近光控制时，该
指示灯点亮。
后雾灯指示灯：打开后雾灯时，该指示灯点亮。
位置灯指示灯：打开位置灯时，该指示灯点亮。
泊车紧急制动指示灯：泊车紧急制动激活时，该指示灯点
亮。
{'source': 'data/初赛训练数据集.pdf', 'page': 73}
----------------------------------------------------------------------------------------------------
Document 3:

 + 灯点亮。
陡坡缓降系统故障警告灯：陡坡缓降系统出现故障时，该
警告灯点亮。
安全气囊故障警告灯：安全气囊系统或预紧器系统存在故
障时，该警告灯点亮。
{'source': 'data/初赛训练数据集.pdf', 'page': 72}
----------------------------------------------------------------------------------------------------
Document 4:

 + 仪表和灯光
89
01点击开启/关闭门控灯。
使用超车灯
超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一
次，以提醒前方车辆。
使用危险警告灯
危险警告灯按键
车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危
险警告灯。
说明！
□发生碰撞情况下，危险警告灯也将自动点亮。
{'source': 'data/初赛训练数据集.pdf', '

# 评估RAG

[使用您的测试集进行评估](https://docs.ragas.io/en/stable/getstarted/evaluation.html)

数据集包含以下列：

- question: list[str] - 这些是将评估您的 RAG 管道的问题。
- answer: list[str] - 您的 RAG 管道生成的答案。
- context: list[list[str]] - 传递到 LLM 来回答问题的上下文。
- ground_truth: list[str] - 问题的真实答案。


In [20]:
questions = ["怎么打开危险警告灯？",
            "车辆如何保养？",
            "靠背太热怎么办？"]

ground_truths = ["危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。",
            "为了保持车辆处于最佳状态，建议您定期关注车辆状态，包括定期保养、洗车、内部清洁、外部清洁、轮胎的保养、低压蓄电池的保养等。",
            "您好，如果您的座椅靠背太热，可以尝试关闭座椅加热功能。在多媒体显示屏上依次点击空调开启按键→座椅→加热，在该界面下可以关闭座椅加热。"]

In [21]:
# 生成 answers 和 contexts
answers = []
contexts = []

for question in questions:
    print(question)
    response = chain(question)
    print(response['result'], "\n")
    answers.append(response['result'])
    contexts.append([doc.page_content for doc in response['source_documents']])

怎么打开危险警告灯？
要打开危险警告灯，您需要按下车辆上的危险警告灯按键。当车辆遇到交通事故或其他紧急情况时，按下该按键即可启用危险警告灯。另外，如果发生碰撞情况，危险警告灯也会自动点亮。 

车辆如何保养？
根据提供的信息，以下是一些关于车辆保养的建议：
1. 清洁外部：在清洗车辆外部前，请关闭所有车窗和天窗。避免使用含酸清洁剂，强碱性肥皂、强化学性清洗剂、汽油或溶剂清洗车辆。冬季道路撒盐的地区应定期清洁车底，清洗完车辆后擦拭干净车辆表面。
2. 存放车辆：车辆应存放在阴凉通风、清洁干燥的环境下，避免长期停放在封闭潮湿环境中。
3. 更换遥控钥匙电池：如果遥控钥匙电池电量耗尽，应及时更换新电池。
4. 清洁显示器/屏幕：经常用超细纤维布清洁显示器，避免液体和水汽接触。
5. 保养项目：根据时间和里程确定保养周期，包括更换发动机机油和机油滤芯、空气滤芯、火花塞等。
6. 保养轮胎：定期检查轮胎是否损坏，避免接触润滑脂、润滑油和燃油。

这些是一些基本的车辆保养建议，具体的保养项目和周期可能会根据车辆型号和使用情况而有所不同。 

靠背太热怎么办？
如果座椅靠背太热，您可以尝试关闭座椅加热功能。根据提供的信息，您可以使用车辆的中央显示屏或Lynk&CoApp来控制前排座椅加热功能的开关。通过关闭加热功能，座椅靠背的温度应该会逐渐降低到更舒适的水平。 



In [23]:
# 构建评估数据集
from datasets import Dataset

evaluate_data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

evaluate_dataset = Dataset.from_dict(evaluate_data)
evaluate_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 3
})

In [24]:
evaluate_dataset[0]

{'question': '怎么打开危险警告灯？',
 'answer': '要打开危险警告灯，您需要按下车辆上的危险警告灯按键。当车辆遇到交通事故或其他紧急情况时，按下该按键即可启用危险警告灯。另外，如果发生碰撞情况，危险警告灯也会自动点亮。',
 'contexts': ['使用危险警告灯 ........................................................ 89',
  '指示灯闪烁。打开危险警告灯开关时，两侧转\n向指示灯均闪烁。\n近光灯指示灯：打开近光灯时，该指示灯点亮。\n远光灯指示灯：打开远光灯时，该指示灯点亮。\n智能远近光控制开启指示灯：打开智能远近光控制时，该\n指示灯点亮。\n后雾灯指示灯：打开后雾灯时，该指示灯点亮。\n位置灯指示灯：打开位置灯时，该指示灯点亮。\n泊车紧急制动指示灯：泊车紧急制动激活时，该指示灯点\n亮。',
  '灯点亮。\n陡坡缓降系统故障警告灯：陡坡缓降系统出现故障时，该\n警告灯点亮。\n安全气囊故障警告灯：安全气囊系统或预紧器系统存在故\n障时，该警告灯点亮。',
  '仪表和灯光\n89\n01点击开启/关闭门控灯。\n使用超车灯\n超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一\n次，以提醒前方车辆。\n使用危险警告灯\n危险警告灯按键\n车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危\n险警告灯。\n说明！\n□发生碰撞情况下，危险警告灯也将自动点亮。',
  '灯常亮。盲点监测系统标定未完成时，该警告灯闪烁。\n低速行人提示音功能故障警告灯：低速行人提示音功能故\n障时，该警告灯点亮。\n查看组合仪表信息\n机舱盖/车门/尾门/安全带提醒界面\n当机舱盖/车门/尾门未关闭或安全带未系时，仪表显示屏将红色高亮\n显示未关闭机舱盖/车门/尾门或未系安全带座椅。\n当车速小于 10km/h时，按下方向盘右侧确认按键。可以隐藏机舱盖/\n车门/尾门/安全带提醒界面。'],
 'ground_truth': '危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。'}


[评估指标](https://docs.ragas.io/en/stable/concepts/metrics/index.html)
- Faithfulness
  - 如果答案中提出的所有主张都可以从给定的上下文中推断出来，则生成的答案被认为是忠实的。
  - answer 和 contexts
- Answer Relevancy
  - 评估指标“答案相关性”重点评估生成的答案与给定提示的相关程度。不完整或包含冗余信息的答案将获得较低分数。
  - answer 和 question
- Context Precision
  - 用于评估 contexts 中存在的所有真实相关项目是否排名较高。理想情况下，所有相关块必须出现在顶层。
  - question 和 contexts
- Context Recall
  - 上下文回忆衡量检索到的上下文与带注释的答案（被视为基本事实）的一致程度。
  - ground truth 和 contexts
---

- RAGAS评估框架评估了RAG管道的两个主要组件：
    - Retriever 检索器
    - Generator 生成器

---

- 与评估 Retrieval 相关的指标如下：
    - Context Precision。question 和 contexts
    - Context Recall。ground truth 和 contexts
- 与评估 Generation 相关的指标如下：
  - Faithfulness。answer 和 contexts
  - Answer Relevancy。answer 和 question

In [25]:
# 导入指标
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


# 开始评估
from ragas import evaluate

evaluate_result = evaluate(
    evaluate_dataset, 
    metrics=[
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ]
)
evaluate_result

Evaluating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:26<00:00,  2.17s/it]


{'faithfulness': 0.6000, 'answer_relevancy': 0.6278, 'context_recall': 1.0000, 'context_precision': 0.3167}

In [26]:
df_evaluate_result = evaluate_result.to_pandas()
df_evaluate_result

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision
0,怎么打开危险警告灯？,要打开危险警告灯，您需要按下车辆上的危险警告灯按键。当车辆遇到交通事故或其他紧急情况时，按下...,[使用危险警告灯 ........................................,危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。,1.0,0.991388,1.0,0.25
1,车辆如何保养？,根据提供的信息，以下是一些关于车辆保养的建议：\n1. 清洁外部：在清洗车辆外部前，请关闭所...,[保养和维护\n317\n注意！\n■在清洗车辆外部前，请记得关闭所有车窗和天窗。\n■请勿...,为了保持车辆处于最佳状态，建议您定期关注车辆状态，包括定期保养、洗车、内部清洁、外部清洁、轮...,0.8,0.892005,1.0,0.70
2,靠背太热怎么办？,如果座椅靠背太热，您可以尝试关闭座椅加热功能。根据提供的信息，您可以使用车辆的中央显示屏或L...,[■展开：以相反方向推动座椅靠背直到完全展开。\n警告！\n■严禁让乘客乘坐在后备厢中。\n...,您好，如果您的座椅靠背太热，可以尝试关闭座椅加热功能。在多媒体显示屏上依次点击空调开启按键→...,0.0,0.000000,1.0,0.00


In [27]:
df_evaluate_result.to_excel(f'data/{index_name}_eval_result.xlsx', index=False)